Due to an unforeseeable errors in the notebook environment, I encountered errors when attempting to import the exact same libaries as here on another notebook. As a result, I have consolidated everything into this notebook. 
First, In order to create the necessary dataset, I ran the relevant code which subsequently stored the data in the designated data folder. 
Following this, I proceeded to run the evaluation using both BERT and BiLSTM. 

In [4]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

In [6]:
import numpy as np
import json

## Dataset Creation
#### the data are stored in Data/ folder

### Subject_Switching

In [322]:
test_examples = [ "John loves Mary.",
                 "Mary loves John.",
                 "John hates Mary.",
                 "Mary hates John.",]
#I followed this guide to generate my instances https://www.kaggle.com/code/juliusalphonso/filling-in-masked-words-with-roberta
def test_generator(test_examples, mask_indices, outputfile):
    unmask = pipeline('fill-mask')
    unmask.tokenizer.mask_token

    challange_sentences = []
    for example in test_examples:
        example_words = example.split()
        for mask_index in mask_indices:
            masked_example = example_words.copy()
            masked_example[mask_index] = unmask.tokenizer.mask_token
            masked_text = " ".join(masked_example)
            generated = unmask(masked_text)
            for result in generated:
                new_example = example_words.copy()
                new_example[mask_index] = result['token_str']
                challange_sentences.append(new_example)

    extended_list = []
    for sentence in challange_sentences:
        verb = sentence[1].strip()
        sentence[-1] = sentence[-1].strip()
        # Add a period at the end of the sentence if it's missing
        # Check if the last token ends with a period
        if not sentence[-1].endswith('.'):
            # Add a period to the last token
            sentence[-1] = sentence[-1] + '.'
        
        
        output_json = {
            "sentence": " ".join(sentence),
            "labels": [
                {
                    "verb": verb,
                    "tags": ["B-ARG0", "B-V", "B-ARG1", "O"]
                }
            ]
        }
        extended_list.append(output_json)

    with open(outputfile, 'w') as outfile:
        json.dump(extended_list, outfile)


unmask = create_pipeline()
outputfile= 'Data/Subject_Switching.json'
# Mask the 1st token
mask_indices = [0,2]
test_generator(test_examples, mask_indices, outputfile)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


### disambiguous noun

In [316]:
import os
from transformers import pipeline
import json

test_examples = [
    ["A", "fight", "with", "spiders", "makes", "you", "brave", "."],
    ["A", "walk", "with", "spiders", "makes", "you", "smart", "."]
]

#I followed this guide to generate my instances https://www.kaggle.com/code/juliusalphonso/filling-in-masked-words-with-roberta



def test_generator(test_examples, mask_indices, outputfile):
    unmask = pipeline('fill-mask')
    unmask.tokenizer.mask_token

    challange_sentences = []
    for example in test_examples:
        for mask_index in mask_indices:
            masked_example = example.copy()
            masked_example[mask_index] = unmask.tokenizer.mask_token
            masked_text = " ".join(masked_example)
            generated = unmask(masked_text)
            for result in generated:
                new_example = example.copy()
                new_example[mask_index] = result['token_str']
                challange_sentences.append(new_example)

    extended_list = []
    for sentence in challange_sentences:
        verb = sentence[4].strip()
        # Remove any extra space before the period
        sentence[-1] = sentence[-1].strip()

        # Add a period at the end of the sentence if it's missing
        # Check if the last token ends with a period
        if not sentence[-1].endswith('.'):
            # Add a period to the last token
            sentence[-1] = sentence[-1] + '.'
        output_json = {
            "sentence": " ".join(sentence),
            "labels": [
                {
                    "verb": verb,
                    "tags": ["B-ARG0", "I-ARG0", "I-ARG0", "I-ARG0", "B-V", "B-ARG1", "I-ARG1", "O"]
                }
            ]
        }
        extended_list.append(output_json)

    with open(outputfile, 'w') as outfile:
        json.dump(extended_list, outfile)


unmask = create_pipeline()
outputfile= 'Data/disambiguous_noun.json'
# Mask the 1st token
mask_indices = [1]
test_generator(test_examples, mask_indices, outputfile)



No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


### passive_voice

In [342]:
import os
from transformers import pipeline
import json

test_examples = [
    "The cake was baked by the chef.",
    "The message was delivered by the courier.",
    "The computer was repaired by the technician.",
    "The book was written by the author.",
    "The painting was created by the artist."]

def test_generator(test_examples, mask_indices, outputfile):
    unmask = pipeline('fill-mask')
    unmask.tokenizer.mask_token

    challange_sentences = []
    for example in test_examples:
        example_words = example.split()
        for mask_index in mask_indices:
            masked_example = example_words.copy()
            masked_example[mask_index] = unmask.tokenizer.mask_token
            masked_text = " ".join(masked_example)
            generated = unmask(masked_text)
            for result in generated:
                new_example = example_words.copy()
                new_example[mask_index] = result['token_str']
                challange_sentences.append(new_example)

    extended_list = []
    for sentence in challange_sentences:
        verb1 = sentence[2].strip()
        verb2 = sentence[3].strip()

        # Remove any extra space before the period
        sentence[-1] = sentence[-1].strip()

        # Add a period at the end of the sentence if it's missing
        # Check if the last token ends with a period
        if not sentence[-1].endswith('.'):
            # Add a period to the last token
            sentence[-1] = sentence[-1] + '.'
        
        output_json = {
            "sentence": " ".join(sentence),
            "labels": [
                {
                    "verb": verb1,
                    "tags": ['O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O']
                },
                {
                    "verb": verb2,
                    "tags": ['B-ARG1', 'I-ARG1', 'O', 'B-V', 'B-ARG0', 'I-ARG0', 'I-ARG0', 'O']
                }
            ]
        }
        extended_list.append(output_json)

    with open(outputfile, 'w') as outfile:
        json.dump(extended_list, outfile, indent=4)

unmask = pipeline('fill-mask')
outputfile = 'Data/passive_voice.json'
# Mask the 1st, 3rd, and 6th token
mask_indices = [1, 3, 6]
test_generator(test_examples, mask_indices, outputfile)


No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


### compund_subjects

In [344]:

test_examples = [ "The  cat and dog chased the ball.",
                 "The  mom and dad helped the child.",
                 "The boy and  girl played the game.",
                 "The sun and  rays lit the sky.",
                "The chef and  crew made the dish."]
#I followed this guide to generate my instances https://www.kaggle.com/code/juliusalphonso/filling-in-masked-words-with-roberta
def test_generator(test_examples, mask_indices, outputfile):
    unmask = pipeline('fill-mask')
    unmask.tokenizer.mask_token

    challange_sentences = []
    for example in test_examples:
        example_words = example.split()
        for mask_index in mask_indices:
            masked_example = example_words.copy()
            masked_example[mask_index] = unmask.tokenizer.mask_token
            masked_text = " ".join(masked_example)
            generated = unmask(masked_text)
            for result in generated:
                new_example = example_words.copy()
                new_example[mask_index] = result['token_str']
                challange_sentences.append(new_example)

    extended_list = []
    for sentence in challange_sentences:
        verb = sentence[4].strip()
        sentence[-1] = sentence[-1].strip()
        # Add a period at the end of the sentence if it's missing
        # Check if the last token ends with a period
        if not sentence[-1].endswith('.'):
            # Add a period to the last token
            sentence[-1] = sentence[-1] + '.'
        
        
        output_json = {
            "sentence": " ".join(sentence),
            "labels": [
                {
                    "verb": verb,
                    "tags": ["B-ARG0", "I-ARG0", "I-ARG0", "I-ARG0", "B-V", "B-ARG1", "I-ARG1", "O"]
                }
            ]
        }
        extended_list.append(output_json)

    with open(outputfile, 'w') as outfile:
        json.dump(extended_list, outfile, indent=4)


unmask = create_pipeline()
outputfile= 'Data/compund_subjects.json'
# Mask the 1st token
mask_indices = [1,3]
test_generator(test_examples, mask_indices, outputfile)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


### Dative Construction Test

In [341]:


test_examples = [ "Jessica taught her son the alphabet.",
                 "Kevin brought his girlfriend some flowers.",
                 "Hannah showed her friends a video.",
                 "Mark bought his son a toy."]
#I followed this guide to generate my instances https://www.kaggle.com/code/juliusalphonso/filling-in-masked-words-with-roberta
def test_generator(test_examples, mask_indices, outputfile):
    unmask = pipeline('fill-mask')
    unmask.tokenizer.mask_token

    challange_sentences = []
    for example in test_examples:
        example_words = example.split()
        for mask_index in mask_indices:
            masked_example = example_words.copy()
            masked_example[mask_index] = unmask.tokenizer.mask_token
            masked_text = " ".join(masked_example)
            generated = unmask(masked_text)
            for result in generated:
                new_example = example_words.copy()
                new_example[mask_index] = result['token_str']
                challange_sentences.append(new_example)

    extended_list = []
    for sentence in challange_sentences:
        verb = sentence[1].strip()
        sentence[-1] = sentence[-1].strip()
        # Add a period at the end of the sentence if it's missing
        # Check if the last token ends with a period
        if not sentence[-1].endswith('.'):
            # Add a period to the last token
            sentence[-1] = sentence[-1] + '.'
        
        
        output_json = {
            "sentence": " ".join(sentence),
            "labels": [
                {
                    "verb": verb,
                    "tags": ["B-ARG0", "B-V", "B-ARG2", "I-ARG2", "B-ARG1", "I-ARG1", "O"]
                }
            ]
        }
        extended_list.append(output_json)

    with open(outputfile, 'w') as outfile:
        json.dump(extended_list, outfile, indent=4)


unmask = create_pipeline()
outputfile= 'Data/dative.json'
# Mask the 1st token
mask_indices = [1]
test_generator(test_examples, mask_indices, outputfile)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


### loading the models

In [7]:
bert = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")


Output()

error loading _jsonnet (this is expected on Windows), treating C:\Users\fapay\AppData\Local\Temp\tmplqcj38mt\config.json as plain json


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\fapay\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v

Output()

error loading _jsonnet (this is expected on Windows), treating C:\Users\fapay\AppData\Local\Temp\tmpe4r9l5fb\config.json as plain json


In [8]:
bilstm = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

error loading _jsonnet (this is expected on Windows), treating C:\Users\fapay\AppData\Local\Temp\tmp7d2nt4j4\config.json as plain json


### loading the files

In [331]:
import json

def load_test_dataset(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

passive_voice = load_test_dataset('Data/passive_voice.json')
disambiguous_noun = load_test_dataset('Data/disambiguous_noun.json')
dative  = load_test_dataset('Data/dative.json')
compund_subjects = load_test_dataset('Data/compund_subjects.json')
Subject_Switching = load_test_dataset('Data/Subject_Switching.json')

#### evulation 

In [305]:
def evaluate_model(predictor, test_dataset):
    total_items = 0
    correct_items = 0
    incorrect_items = 0
    incorrect_predictions = []

    for test_case in test_dataset:
        sentence = test_case['sentence']
        expected_outputs = test_case['labels']
        predicted_output = predictor.predict(sentence)

        if not expected_outputs or not predicted_output['verbs']:
            continue  # Skip empty lists

        matched_verbs = []
        for expected in expected_outputs:
            total_items += 1
            found_match = False
            for predicted in predicted_output['verbs']:
                if expected['verb'] == predicted['verb'] and expected['tags'] == predicted['tags']:
                    found_match = True
                    matched_verbs.append(predicted)
                    break

            if found_match:
                correct_items += 1
            else:
                incorrect_items += 1
                incorrect_predictions.append({
                    "sentence": sentence,
                    "expected_output": expected,
                    "predicted_output": predicted_output['verbs']
                })

        unmatched_verbs = [verb for verb in predicted_output['verbs'] if verb not in matched_verbs]
        for unmatched_verb in unmatched_verbs:
            total_items += 1
            incorrect_items += 1
            incorrect_predictions.append({
                "sentence": sentence,
                "expected_output": None,
                "predicted_output": unmatched_verb
            })

    error_rate = incorrect_items / total_items
    accuracy = correct_items / total_items

    return error_rate, accuracy, incorrect_predictions

In [320]:
import pandas as pd
def present_results(incorrect_predictions):
    data = []

    for prediction in incorrect_predictions:
        sentence = prediction['sentence']
        expected_output = prediction['expected_output']
        predicted_output = prediction['predicted_output']

        if expected_output is not None:
            correct_tags = expected_output['tags']
        else:
            correct_tags = None

        incorrect_preds = [pred['tags'] for pred in predicted_output if isinstance(pred, dict) and pred['tags'] != correct_tags]

        if incorrect_preds:
            data.append({
                'Sentence': sentence,
                'Correct Tags': correct_tags,
                'Predicted Tags': incorrect_preds[0]  # Only add the first incorrect prediction
            })

    df = pd.DataFrame(data)
    return df


In [ ]:
#saving the output 

In [351]:
#passive_Voice
pv_bert_error_rate, pv_bert_accuracy, pv_bert_incorrect = evaluate_model(bert, passive_voice)
pv_biltsm_error_rate, pv_bilstm_accuracy, pv_bilstm_incorrect = evaluate_model(bilstm, passive_voice)

In [352]:
results_df = present_results(pv_bert_incorrect)
results_df.to_csv('result/passive_voice_bert.tsv', sep='\t', index=False)
results_df = present_results(pv_bilstm_incorrect)
results_df.to_csv('result/passive_voice_bilstm.tsv', sep='\t', index=False)

In [355]:
#disambiguous_noun
dn_bert_error_rate, dn_bert_accuracy, dn_bert_incorrect = evaluate_model(bert, disambiguous_noun)
dn_biltsm_error_rate, dn_bilstm_accuracy, dn_bilstm_incorrect = evaluate_model(bilstm, disambiguous_noun)

In [359]:
results_df = present_results(dn_bert_incorrect)
results_df.to_csv('result/disambiguous_noun_bert.tsv', sep='\t', index=False)
results_df = present_results(dn_bilstm_incorrect)
results_df.to_csv('result/disambiguous_noun_bilstm.tsv', sep='\t', index=False)

In [356]:
#dative
dative_bert_error_rate, dative_bert_accuracy, dative_bert_incorrect = evaluate_model(bert, dative)
dative_biltsm_error_rate, dative_bilstm_accuracy, dative_bilstm_incorrect = evaluate_model(bilstm, dative)

In [360]:
results_df = present_results(dative_bert_incorrect)
results_df.to_csv('result/dative_bert.tsv', sep='\t', index=False)
results_df = present_results(dative_bilstm_incorrect)
results_df.to_csv('result/dative_bilstm.tsv', sep='\t', index=False)

In [358]:
#compund_subjects
cs_bert_error_rate, cs_bert_accuracy, cs_bert_incorrect = evaluate_model(bert, compund_subjects)
cs_biltsm_error_rate, cs_bilstm_accuracy, cs_bilstm_incorrect = evaluate_model(bilstm, compund_subjects)


In [361]:
results_df = present_results(cs_bert_incorrect)
results_df.to_csv('result/compund_subjects_bert.tsv', sep='\t', index=False)
results_df = present_results(cs_bilstm_incorrect)
results_df.to_csv('result/compund_subjects_bilstm.tsv', sep='\t', index=False)

In [371]:
#Subject_Switching
ss_bert_error_rate, ss_bert_accuracy, ss_bert_incorrect = evaluate_model(bert, Subject_Switching)
ss_biltsm_error_rate, ss_bilstm_accuracy, ss_bilstm_incorrect = evaluate_model(bilstm, Subject_Switching)


In [372]:
results_df = present_results(ss_bert_incorrect)
results_df.to_csv('result/Subject_Switching_bert.tsv', sep='\t', index=False)
results_df = present_results(ss_bilstm_incorrect)
results_df.to_csv('result/Subject_Switching_bilstm.tsv', sep='\t', index=False)

In [374]:
### inspection of the result here, for instnace:
dative_bert_incorrect

[{'sentence': 'Kevin  bought his girlfriend some flowers.',
  'expected_output': {'verb': 'bought',
   'tags': ['B-ARG0', 'B-V', 'B-ARG2', 'I-ARG2', 'B-ARG1', 'I-ARG1', 'O']},
  'predicted_output': [{'verb': 'bought',
    'description': '[ARG0: Kevin] [V: bought] [ARG4: his girlfriend] [ARG1: some flowers] .',
    'tags': ['B-ARG0', 'B-V', 'B-ARG4', 'I-ARG4', 'B-ARG1', 'I-ARG1', 'O']}]},
 {'sentence': 'Kevin  bought his girlfriend some flowers.',
  'expected_output': None,
  'predicted_output': {'verb': 'bought',
   'description': '[ARG0: Kevin] [V: bought] [ARG4: his girlfriend] [ARG1: some flowers] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG4', 'I-ARG4', 'B-ARG1', 'I-ARG1', 'O']}},
 {'sentence': 'Mark  bought his son a toy.',
  'expected_output': {'verb': 'bought',
   'tags': ['B-ARG0', 'B-V', 'B-ARG2', 'I-ARG2', 'B-ARG1', 'I-ARG1', 'O']},
  'predicted_output': [{'verb': 'bought',
    'description': '[ARG0: Mark] [V: bought] [ARG4: his son] [ARG1: a toy] .',
    'tags': ['B-ARG0', 'B-V',